In [ ]:
from local.imports import *
from local.test import *
from local.core import *
from local.layers import *
from local.data.all import *
from local.notebook.showdoc import show_doc
from local.optimizer import *
from local.learner import *
from local.text.data import *
from local.text.models.core import *
from local.text.models.awdlstm import *
from local.callback.rnn import *
from local.callback.all import *

# Integration test on Wikitext-2

> Training a Language Model on WT2

In [ ]:
path = untar_data(URLs.WIKITEXT_TINY)

The dataset comes with all the wrticles concatenated. We split them to be able to shuffle at the beginning of each epoch.

In [ ]:
def istitle(line):
    return len(re.findall(r'^ = [^=]* = $', line)) != 0

def read_file(filename):
    articles = L()
    with open(filename, encoding='utf8') as f:
        lines = f.readlines()
    current_article = ''
    for i,line in enumerate(lines):
        current_article += line
        if i < len(lines)-2 and lines[i+1] == ' \n' and istitle(lines[i+2]):
            articles.append(current_article.split(' '))
            current_article = ''
    articles.append(current_article.split(' '))
    return articles

Then we put our list of tokenized texts together in an `LM_Dataset`. It will return tuples of sequences of `seq_len`, with the second sequence between the first one shifted by one on the right.

In [ ]:
bs,sl = 104,72
train = LM_Dataset(read_file(path/'train.txt'), bs=bs, seq_len=sl, shuffle=True)
valid = LM_Dataset(read_file(path/'valid.txt'), bs=bs, seq_len=sl)

In [ ]:
print(train[0])

(['', '\n', '=', 'Geopyxis', 'carbonaria', '=', '\n', '\n', 'Geopyxis', 'carbonaria', 'is', 'a', 'species', 'of', 'fungus', 'in', 'the', 'genus', 'Geopyxis', ',', 'family', '<unk>', '.', 'First', 'described', 'to', 'science', 'in', '1805', ',', 'and', 'given', 'its', 'current', 'name', 'in', '1889', ',', 'the', 'species', 'is', 'commonly', 'known', 'as', 'the', 'charcoal', 'loving', 'elf', '@-@', 'cup', ',', 'dwarf', '<unk>', 'cup', ',', '<unk>', '<unk>', 'cup', ',', 'or', 'pixie', 'cup', '.', 'The', 'small', ',', '<unk>', '@-@', 'shaped', 'fruitbodies', 'of', 'the'], ['\n', '=', 'Geopyxis', 'carbonaria', '=', '\n', '\n', 'Geopyxis', 'carbonaria', 'is', 'a', 'species', 'of', 'fungus', 'in', 'the', 'genus', 'Geopyxis', ',', 'family', '<unk>', '.', 'First', 'described', 'to', 'science', 'in', '1805', ',', 'and', 'given', 'its', 'current', 'name', 'in', '1889', ',', 'the', 'species', 'is', 'commonly', 'known', 'as', 'the', 'charcoal', 'loving', 'elf', '@-@', 'cup', ',', 'dwarf', '<unk>', 

We can then wrap our `LM_Dataset`s in a `TfmdList` to apply the `Numericalize` transform. We can't use a `TfmdDS` because our elements are already tuples and `TfmdDS` is there to create such tuples from individual items. Since we already have tuples, we specify `as_item=False`.

In [ ]:
count = Counter([p for t in train.ds for p in t])
vocab = make_vocab(count)
train_ds = TfmdList(train, tfms=Numericalize(vocab), as_item=False, wrap_l=False)
valid_ds = TfmdList(valid, tfms=Numericalize(vocab), as_item=False, wrap_l=False)

Last but not least, we need to use a special sampler that will make sure we ask for the correct sequences to form a batch: in the first batch we don't want the sequences 0,1,2,3... (they are contiguous in the source obtained by concatenating all texts) but the sequences 0,`num_batches`,`2*num_batches`,...

In [ ]:
train_dl = TfmdDL(train_ds, bs=bs, sampler=LM_Sampler(train), tfms=Cuda(), num_workers=0)
valid_dl = TfmdDL(valid_ds, bs=bs, sampler=LM_Sampler(valid), tfms=Cuda(), num_workers=0)

In [ ]:
dbch = DataBunch(train_dl, valid_dl)
dbch.show_batch()

index,text,text_
0,"\n = Geopyxis carbonaria = \n \n Geopyxis carbonaria is a species of fungus in the genus Geopyxis , family <unk> . First described to science in 1805 , and given its current name in 1889 , the species is commonly known as the charcoal loving elf @-@ cup , dwarf <unk> cup , <unk> <unk> cup , or pixie cup . The small , <unk> @-@ shaped fruitbodies of the","\n = Geopyxis carbonaria = \n \n Geopyxis carbonaria is a species of fungus in the genus Geopyxis , family <unk> . First described to science in 1805 , and given its current name in 1889 , the species is commonly known as the charcoal loving elf @-@ cup , dwarf <unk> cup , <unk> <unk> cup , or pixie cup . The small , <unk> @-@ shaped fruitbodies of the fungus"
1,"Creek is in a sandstone and shale mountain region , it has a relatively low capacity to neutralize added acid . This makes it especially vulnerable to increased <unk> from acid rain , which poses a threat to the long term health of the plants and animals in the creek . The total <unk> ( TA ) is a measure of the capacity of water to neutralize acid , with a larger","is in a sandstone and shale mountain region , it has a relatively low capacity to neutralize added acid . This makes it especially vulnerable to increased <unk> from acid rain , which poses a threat to the long term health of the plants and animals in the creek . The total <unk> ( TA ) is a measure of the capacity of water to neutralize acid , with a larger TA"
2,"the homes of wealthy Jews . \n Finally her soliciting paid off and in 1881 , Rosebery was offered a government position acceptable to him , that of Under Secretary at the Home Office with special responsibility for Scotland . He had sought the position feeling that Scotland was neglected by the Liberal Government who were more interested in Ireland . However , immediately upon assuming the job he began to demand","homes of wealthy Jews . \n Finally her soliciting paid off and in 1881 , Rosebery was offered a government position acceptable to him , that of Under Secretary at the Home Office with special responsibility for Scotland . He had sought the position feeling that Scotland was neglected by the Liberal Government who were more interested in Ireland . However , immediately upon assuming the job he began to demand a"
3,"and Sweden , and the Danish and Swedish governments threatened to terminate the SAS agreement . On 25 November 1966 , with 82 against 62 votes , the Norwegian Parliament voted to allow Braathens SAFE to fly to Bodø and Tromsø as an extension of the West Coast route from 1 April 1967 . \n On 3 March 1967 , parliament decided to build four short take @-@ off and landing airports","Sweden , and the Danish and Swedish governments threatened to terminate the SAS agreement . On 25 November 1966 , with 82 against 62 votes , the Norwegian Parliament voted to allow Braathens SAFE to fly to Bodø and Tromsø as an extension of the West Coast route from 1 April 1967 . \n On 3 March 1967 , parliament decided to build four short take @-@ off and landing airports along"
4,"the outfitting center for Colorado gold <unk> headed for <unk> Peak in 1859 , 268 steamboats arrived at Omaha between March and November . \n With railroads becoming the dominant form of long @-@ range shipping and passenger travel in the early 1870s , <unk> like those in Omaha became obsolete . However , as late at 1949 the steamship Avalon was letting passengers in Omaha , before becoming one of the","outfitting center for Colorado gold <unk> headed for <unk> Peak in 1859 , 268 steamboats arrived at Omaha between March and November . \n With railroads becoming the dominant form of long @-@ range shipping and passenger travel in the early 1870s , <unk> like those in Omaha became obsolete . However , as late at 1949 the steamship Avalon was letting passengers in Omaha , before becoming one of the famous"
5,"of February 2016 , Vistara has a share of 2 % in the domestic carrier market . \n \n = = Corporate 

In [ ]:
config = awd_lstm_lm_config.copy()
config.update({'input_p': 0.6, 'output_p': 0.4, 'weight_p': 0.5, 'embed_p': 0.1, 'hidden_p': 0.2})
model = get_language_model(AWD_LSTM, len(vocab), config=config)

In [ ]:
opt_func = partial(Adam, wd=0.1, eps=1e-7)
cb_funcs = [partial(MixedPrecision, clip=0.1), partial(RNNTrainer, alpha=3, beta=2)]

In [ ]:
learn = Learner(model, dbch, loss_func=CrossEntropyLossFlat(), opt_func=opt_func, cb_funcs=cb_funcs)

In [ ]:
%prun learn.fit_one_cycle(1, 5e-3, moms=(0.8,0.7,0.8), div=10)

epoch,train_loss,valid_loss,time
0,5.618763,5.228721,02:34
